In [ ]:
## Initiales Importieren. Bevor das getan wird: `./panda.py categorize` ausführen!
import datetime
import pandas as pd
from panda import load_pc, add_cat
pc = load_pc()
pc = add_cat(pc)
pc.columns

In [ ]:
## Finden von nicht-kategorisierten Abbuchungen auf den Konten 'giro', 'gesa' und 'common' im Jahr 2024

# Kopie des DataFrames erstellen
df = pc.loc[(pc.book_date.dt.year == 2024) & (pc.transfer_category.isna())]
df = df.copy()

# 'amount_type' setzen basierend auf dem Betrag
df.loc[df['amount'] > 0, 'amount_type'] = 'Gutschrift'
df.loc[df['amount'] <= 0, 'amount_type'] = 'Abbuchung'

# Relevante Spalten auswählen
cols = ['account', 'book_date', 'party', 'purpose', 'amount', 'cat']

# Filter für Abbuchungen und die Konten 'giro' und 'common'
df = df.loc[
    (df['amount_type'] == 'Abbuchung') & (df['account'].isin(['giro', 'gesa', 'common']))
    ][cols].sort_values(by='amount', ascending=True)

# Nur Einträge ohne Kategorie anzeigen
filtered_df = df.loc[df['cat'].isna()]
print(filtered_df['amount'].sum())
filtered_df

In [ ]:
## Innerhalb der nicht-kategorisierten Abbuchungen, kumulative Summe der Top-X Beträge berechnen und plotten
import matplotlib.pyplot as plt

# Calculate the cumulative sum of the amounts
filtered_df = filtered_df.sort_values(by='amount', ascending=True)
filtered_df['cumulative_sum'] = filtered_df['amount'].cumsum()

# Plot the cumulative sum
plt.figure(figsize=(20, 12))
plt.plot(range(1, len(filtered_df) + 1), filtered_df['cumulative_sum'])
plt.xlabel('Number of Items')
plt.ylabel('Cumulative Sum of Amount')
plt.title('Cumulative Sum of Top-x Uncategorized Expenses')
plt.grid(True)
plt.show()

In [ ]:
## Gesamteinnahmen 2024
income_df = pc.loc[
    (pc.book_date.dt.year == 2024) &
    (pc['cat'].isin(['einnahmen::gehalt::andreas', 'einnahmen::gehalt::gesa', 'einnahmen::dividende']))
    ]
income_df

In [ ]:
## Einkommensuebersicht 2024
def generate_income_overview(income_df: pd.DataFrame) -> pd.DataFrame:
    # Sum by category
    category_sum = income_df.groupby('cat')['amount'].sum()

    # Overall sum
    overall_sum = income_df['amount'].sum()

    # Combine results into a DataFrame
    overview_df = category_sum.reset_index().rename(columns={'amount': 'category_sum'})
    overview_df.loc[len(overview_df)] = ['Overall Sum', overall_sum]

    return overview_df
# Generate the overview
income_overview_df = generate_income_overview(income_df)
income_overview_df

In [ ]:
## Filter für alle Ausgaben im Jahr 2024
expenses_df = pc.loc[
    (pc.book_date.dt.year == 2024) &  # Nur Buchungen aus dem Jahr 2024
    (~pc['cat'].str.startswith('intern', na=False)) &  # "intern"-Kategorie ausschließen
    (~pc['cat'].str.startswith('einnahmen', na=False)) &  # "einnahmen"-Kategorie ausschließen
    (pc['transfer_category'].isna()) &  # transfer_category muss NaN sein
    (pc['account'].isin(['giro', 'common', 'gesa']))  # Nur bestimmte Konten berücksichtigen
    ]

print(expenses_df['amount'].sum())
expenses_df

In [ ]:
## Ausgaben nach Kategorie, Betragstyp und Konto gruppieren
def generate_expense_overview(expenses_df: pd.DataFrame) -> pd.DataFrame:
    # Replace NaN in 'cat' with 'Uncategorized'
    expenses_df['cat'] = expenses_df['cat'].fillna('Uncategorized')

    # Sum by category and account
    category_account_sum = expenses_df.groupby(['cat', 'account'])['amount'].sum().unstack(fill_value=0)

    # Sum by category across all accounts
    category_sum = expenses_df.groupby('cat')['amount'].sum()

    # Overall sum across all accounts
    overall_sum = expenses_df['amount'].sum()

    # Combine results into a DataFrame
    overview_df = category_sum.reset_index().rename(columns={'amount': 'category_sum'})
    overview_df['giro'] = overview_df['cat'].map(category_account_sum.get('giro', {}))
    overview_df['gesa'] = overview_df['cat'].map(category_account_sum.get('gesa', {}))
    overview_df['common'] = overview_df['cat'].map(category_account_sum.get('common', {}))

    # Add overall sum as a final row
    overall_row = pd.DataFrame([{
        'cat': 'Overall Sum',
        'category_sum': overall_sum,
        'giro': category_account_sum.loc[:, 'giro'].sum() if 'giro' in category_account_sum else 0,
        'gesa': category_account_sum.loc[:, 'gesa'].sum() if 'gesa' in category_account_sum else 0,
        'common': category_account_sum.loc[:, 'common'].sum() if 'common' in category_account_sum else 0,
    }])

    overview_df = pd.concat([overview_df, overall_row], ignore_index=True)

    return overview_df

# Generate the overview
expense_overview_df = generate_expense_overview(expenses_df)
expense_overview_df

In [ ]:
pc[(pc.account == "giro") & (pc.amount > 0) & (pc.book_date.dt.year == 2024)]

In [ ]:
## Anwaltskosten
df['book_date'].min()
df = pc
filtered_df = df[
    df['party'].str.contains('KNH|zirngibl', case=False, na=False) |
    df['purpose'].str.contains('KNH|zirngibl', case=False, na=False)
    ]
filtered_df

In [ ]:
## Anwaltskosten #2
filtered_df = df[
    df['cat'].str.startswith('anwalt', na=False) |
    df['purpose'].str.contains('luig', case=False, na=False) |
    df['party'].str.contains('liu', case=False, na=False)
    ]
filtered_df

In [ ]:
# Alle wohnen::putzen Ausgaben fr das Jahr 2023
df = pc[(pc.account == 'common') & (pc.book_date > datetime.datetime(2023, 2, 1, 0,0,0)) & (pc.book_date < datetime.datetime(2024, 2, 1, 0,0,0))]
filtered_df = df[df['cat'] == 'wohnen::putzen']
filtered_df